In [ ]:
from matplotlib import pyplot
import warnings

from summer.utils import ref_times_to_dti

from autumn.core.project import get_project
from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.core.utils.display import pretty_print

In [ ]:
pyplot.style.use("ggplot")
warnings.filterwarnings("ignore")
region = "bhutan"

In [ ]:
p = get_project("sm_sir", region, reload=True)
baseline_params = p.param_set.baseline
model = p.run_baseline_model(baseline_params)
baseline_df = model.get_derived_outputs_df()
model_start_time = ref_times_to_dti(COVID_BASE_DATETIME, [baseline_params["time"]["start"]])[0]
model_end_time = ref_times_to_dti(COVID_BASE_DATETIME, [baseline_params["time"]["end"]])[0]

In [ ]:
def convert_ts_index_to_date(ts):
    ts.index = ref_times_to_dti(COVID_BASE_DATETIME, ts.index)
    return ts

In [ ]:
# Be careful of this:
from autumn.projects.sm_sir.bhutan.bhutan import project

In [ ]:
ts_sets = project.ts_set
ts_set_dates = {k: convert_ts_index_to_date(v) for k, v in ts_sets.items()}

In [ ]:
pretty_print(baseline_params)

In [ ]:
# sc_models = p.run_scenario_models(model, p.param_set.scenarios)
# scenario_dfs = [m.get_derived_outputs_df() for m in sc_models]

In [ ]:
# Define what we want to plot
outputs_to_plot = ["notifications", "infection_deaths", "prop_ever_infected", "cdr"]
plot_end_time = ref_times_to_dti(COVID_BASE_DATETIME, [1000.])[0]

fig = pyplot.figure(figsize=(15, 12))
for i_out, output in enumerate(outputs_to_plot):
    axis = fig.add_subplot(2, 2, i_out + 1)
    
    # Plotting
    if output in ts_set_dates:
        ts_set_dates[output].plot(style='.', label="")
    baseline_df[output].plot(label="baseline")
#     for scenario in range(len(sc_models)):
#         scenario_dfs[scenario][outputs_to_plot[i_out]].plot(label=p.param_set.scenarios[scenario]["description"])
    
    # Display
    axis.set_title(output.replace("_", " "))
    if i_out == 0:
        axis.legend()
    axis.set_xlim([model_start_time, model_end_time])

In [ ]:
ref_times_to_dti(COVID_BASE_DATETIME, [897.])